# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
!pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
exported_csv= "../output_data/cities.csv"

cities_df= pd.read_csv("../output_data/cities.csv")

cities_df.head()

,City,Cloudiness,Country,Date,Humidity,longitude,latitude,Max Temperature,Wind Speed List
0,Leningradskiy,100,RU,1612452564,85,178.4167,69.3833,240.83,2.75
1,Busselton,0,AU,1612452564,64,115.3333,-33.6500,287.19,7.15
2,Barrow,1,US,1612452565,75,-156.7887,71.2906,240.15,7.20
3,Butaritari,100,KI,1612452565,84,172.7902,3.0707,300.00,6.66
4,Dvinskoy,95,RU,1612452565,97,45.1175,62.1535,263.82,1.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
#Config gmaps with gkey
gmaps.configure(api_key=g_key)

cities_df= cities_df.dropna()
humidity = weather_df["Humidity"].astype(float)
locations=cities_df[["latitude","longitude"]]

#Save Lat and Long
locations =weather_df[["latitude", "longitude"]].astype(float)

fig = gmaps.figure()

heater_map = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)

fig.add_layer(heater_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
# Narrow down the DataFrame to find your ideal weather condition. For example:





filtered_cities = cities_df

#   * A max temperature lower than 80 degrees but higher than 70.
filtered_cities= filtered_cities.loc[(filtered_cities["Max Temperature"] < 80) & (filtered_cities["Max Temperature"] > 70)]

#   * Wind speed less than 10 mph.
filtered_cities= filtered_cities.loc[filtered_cities["Wind Speed List"] < 10]      

#   * Zero cloudiness.
filtered_cities=filtered_cities.loc[filtered_cities["Cloudiness"] == 0]

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
filtered_cities = filtered_cities.dropna()

filtered_cities.head(10)

,City,Cloudiness,Country,Date,Humidity,longitude,latitude,Max Temperature,Wind Speed List


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
# hotel_map_df = filtered_cities.loc[:,["City","Country", "Lat", "Lng"]]

# #Add  'Hotel Name' to DataFrame

# hotel_map_df["Hotel Name"] = ""
hotel_df = filtered_cities

hotel_df['Hotel Name'] = ""
#hotel_df.head()
#paramatered dictionary for iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # get requests
    name_address = requests.get(base_url, params=params)
    
    

    # JSONify
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    #  try-except to skip any that are missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             0 non-null      object 
 1   Cloudiness       0 non-null      int64  
 2   Country          0 non-null      object 
 3   Date             0 non-null      int64  
 4   Humidity         0 non-null      int64  
 5   longitude        0 non-null      float64
 6   latitude         0 non-null      float64
 7   Max Temperature  0 non-null      float64
 8   Wind Speed List  0 non-null      float64
 9   Hotel Name       0 non-null      object 
dtypes: float64(4), int64(3), object(3)
memory usage: 0.0+ bytes


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [73]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
final_fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
final_fig.add_layer(heater_map)
final_fig.add_layer(hotel_layer)
# Display figure
final_fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…